<a href="https://colab.research.google.com/github/Bahae9/Kaggle_competitions/blob/main/Loan_Approval_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c playground-series-s4e10
!kaggle datasets download -d chilledwanker/loan-approval-prediction
!unzip loan-approval-prediction.zip
!unzip playground-series-s4e10.zip

  0% 0.00/1.45M [00:00<?, ?B/s]
100% 1.45M/1.45M [00:00<00:00, 77.7MB/s]
Dataset URL: https://www.kaggle.com/datasets/chilledwanker/loan-approval-prediction
License(s): apache-2.0
  0% 0.00/368k [00:00<?, ?B/s]
100% 368k/368k [00:00<00:00, 41.1MB/s]
Archive:  loan-approval-prediction.zip
  inflating: credit_risk_dataset.csv  
Archive:  playground-series-s4e10.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [2]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.1 MB/s eta 0:00:00


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import optuna
from sklearn.model_selection import cross_val_score

In [9]:
challenge_df = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
original_df = pd.read_csv('/content/credit_risk_dataset.csv')
challenge_df.head(2)

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0


In [10]:
original_df.head(2)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2


In [12]:
max_id = challenge_df['id'].max()
original_df['id'] = range(max_id + 1, max_id + 1 + len(original_df))
df = pd.concat([challenge_df, original_df], ignore_index=True)
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91226 entries, 0 to 91225
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          91226 non-null  int64  
 1   person_age                  91226 non-null  int64  
 2   person_income               91226 non-null  int64  
 3   person_home_ownership       91226 non-null  object 
 4   person_emp_length           90331 non-null  float64
 5   loan_intent                 91226 non-null  object 
 6   loan_grade                  91226 non-null  object 
 7   loan_amnt                   91226 non-null  int64  
 8   loan_int_rate               88110 non-null  float64
 9   loan_percent_income         91226 non-null  float64
 10  cb_person_default_on_file   91226 non-null  object 
 11  cb_person_cred_hist_length  91226 non-null  int64  
 12  loan_status                 91226 non-null  int64  
dtypes: float64(3), int64(6), object

In [14]:
def encode(df):
  df = df.replace({'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 0, 'G': -1})
  df = df.replace({'N': 0, 'Y': 1})
  df = pd.get_dummies(df, columns=['person_home_ownership', 'loan_intent'], prefix=['home_ownership', 'loan_intent'])
  return df

In [15]:
df = encode(df)

<ipython-input-14-337341cb87c0>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 0, 'G': -1})
<ipython-input-14-337341cb87c0>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'N': 0, 'Y': 1})


In [18]:
df.fillna(df.mean(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91226 entries, 0 to 91225
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             91226 non-null  int64  
 1   person_age                     91226 non-null  int64  
 2   person_income                  91226 non-null  int64  
 3   person_emp_length              91226 non-null  float64
 4   loan_grade                     91226 non-null  int64  
 5   loan_amnt                      91226 non-null  int64  
 6   loan_int_rate                  91226 non-null  float64
 7   loan_percent_income            91226 non-null  float64
 8   cb_person_default_on_file      91226 non-null  int64  
 9   cb_person_cred_hist_length     91226 non-null  int64  
 10  loan_status                    91226 non-null  int64  
 11  home_ownership_MORTGAGE        91226 non-null  bool   
 12  home_ownership_OTHER           91226 non-null 

In [19]:
test = encode(test)
X_test_pred = test.copy()

<ipython-input-14-337341cb87c0>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1, 'F': 0, 'G': -1})
<ipython-input-14-337341cb87c0>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({'N': 0, 'Y': 1})


In [20]:
X = df.drop(columns=['loan_status'])
y = df['loan_status']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def objective_xgb(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }
    model = XGBClassifier(**params, random_state=42, eval_metric='auc')
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

def objective_lgbm(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }
    model = LGBMClassifier(**params, random_state=42)
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
    }
    model = RandomForestClassifier(**params, random_state=42)
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

def objective_catboost(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 16),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    }
    model = CatBoostClassifier(**params, random_state=42, eval_metric='AUC', verbose=False)
    score = cross_val_score(model, X, y, n_jobs=-1, cv=5, scoring='roc_auc').mean()
    return score

# # Optimize CatBoost hyperparameters
# study_catboost = optuna.create_study(direction='maximize')
# study_catboost.optimize(objective_catboost, n_trials=100)  # Adjust n_trials as needed
# print("Best CatBoost hyperparameters:", study_catboost.best_params)

# # Optimize XGBoost hyperparameters
# study_xgb = optuna.create_study(direction='maximize')
# study_xgb.optimize(objective_xgb, n_trials=30)  # Adjust n_trials as needed
# print("Best XGBoost hyperparameters:", study_xgb.best_params)

# # Optimize LightGBM hyperparameters
# study_lgbm = optuna.create_study(direction='maximize')
# study_lgbm.optimize(objective_lgbm, n_trials=30)  # Adjust n_trials as needed
# print("Best LightGBM hyperparameters:", study_lgbm.best_params)

# Optimize Random Forest hyperparameters
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=100)  # Adjust n_trials as needed
print("Best Random Forest hyperparameters:", study_rf.best_params)

[I 2024-10-03 00:16:37,112] A new study created in memory with name: no-name-13b6144d-b4d2-4c84-9749-e2e3348c82d9
[I 2024-10-03 00:17:09,303] Trial 0 finished with value: 0.9244277743350162 and parameters: {'n_estimators': 193, 'max_depth': 6, 'min_samples_split': 7, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9244277743350162.
[I 2024-10-03 00:17:42,718] Trial 1 finished with value: 0.9266384499119862 and parameters: {'n_estimators': 145, 'max_depth': 10, 'min_samples_split': 19, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.9266384499119862.
[I 2024-10-03 00:18:27,349] Trial 2 finished with value: 0.9195941966350627 and parameters: {'n_estimators': 227, 'max_depth': 8, 'min_samples_split': 20, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.9266384499119862.
[I 2024-10-03 00:19:55,909] Trial 3 finished with value: 0.93396216216538 and parameters: {'n_estimators': 390, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 8}. Best is trial 3 with value: 

KeyboardInterrupt: 

In [ ]:
# Best CatBoost hyperparameters: {'iterations': 389, 'learning_rate': 0.1356181656089208, 'depth': 5, 'l2_leaf_reg': 9.037946029076725, 'border_count': 155, 'random_strength': 3.1441582494546205, 'bagging_temperature': 3.3885699691748004, 'od_type': 'IncToDec', 'od_wait': 15}
# Best XGBoost hyperparameters: {'max_depth': 9, 'learning_rate': 0.08162442747596123, 'n_estimators': 408, 'min_child_weight': 6, 'subsample': 0.9606721160739966, 'colsample_bytree': 0.5246249337091624, 'gamma': 0.9778592845089441, 'reg_alpha': 1.3046095736816625, 'reg_lambda': 9.663682545922917}
# Best LightGBM hyperparameters: {'num_leaves': 225, 'learning_rate': 0.08245968867808884, 'n_estimators': 407, 'max_depth': 6, 'min_child_samples': 46, 'subsample': 0.5469000974065992, 'colsample_bytree': 0.6113458989028802, 'reg_alpha': 7.697039904785172, 'reg_lambda': 5.488324394636368}
# Best Random Forest hyperparameters: {'n_estimators': 269, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10}

# catboost: {'iterations': 1028, 'learning_rate': 0.13974159840348863, 'depth': 6, 'l2_leaf_reg': 4.887113899093811, 'bagging_temperature': 2.2181121823033245, 'min_child_samples': 15, 'subsample': 0.839574196388468}
# rf: {'n_estimators': 465, 'max_depth': 10, 'min_samples_split': 11, 'min_samples_leaf': 9}

In [27]:
# cb_params = {'iterations': 389, 'learning_rate': 0.1356181656089208, 'depth': 5, 'l2_leaf_reg': 9.037946029076725, 'border_count': 155, 'random_strength': 3.1441582494546205, 'bagging_temperature': 3.3885699691748004, 'od_wait': 15}
cb_params = {'iterations': 1028, 'learning_rate': 0.13974159840348863, 'depth': 6, 'l2_leaf_reg': 4.887113899093811, 'bagging_temperature': 2.2181121823033245, 'min_child_samples': 15, 'subsample': 0.839574196388468}
xgb_params = {'max_depth': 9, 'learning_rate': 0.08162442747596123, 'n_estimators': 408, 'min_child_weight': 6, 'subsample': 0.9606721160739966, 'colsample_bytree': 0.5246249337091624, 'gamma': 0.9778592845089441, 'reg_alpha': 1.3046095736816625, 'reg_lambda': 9.663682545922917}
lgbm_params = {'num_leaves': 225, 'learning_rate': 0.08245968867808884, 'n_estimators': 407, 'max_depth': 6, 'min_child_samples': 46, 'subsample': 0.5469000974065992, 'colsample_bytree': 0.6113458989028802, 'reg_alpha': 7.697039904785172, 'reg_lambda': 5.488324394636368}
# rf_params = {'n_estimators': 269, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10}
rf_params = {'n_estimators': 465, 'max_depth': 10, 'min_samples_split': 11, 'min_samples_leaf': 9}

In [38]:
model1 = XGBClassifier(**xgb_params)
model2 = RandomForestClassifier(**rf_params)
model3 = LGBMClassifier(**lgbm_params)
model4 = CatBoostClassifier(**cb_params)
# Create a voting classifier
model = VotingClassifier(estimators=[('xgb', model1), ('rf', model2), ('lgbm', model3), ('cb', model4)], voting='soft')

# Fit the voting classifier
model4.fit(X_train, y_train)

# Make predictions
y_pred = model4.predict_proba(X_test)[:, 1]

# Evaluate the model
roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC Score: {roc_auc}")

0:	learn: 0.5376747	total: 20ms	remaining: 20.5s
1:	learn: 0.4363639	total: 38.3ms	remaining: 19.7s
2:	learn: 0.3681232	total: 56.1ms	remaining: 19.2s
3:	learn: 0.3223002	total: 75.9ms	remaining: 19.4s
4:	learn: 0.2899286	total: 94.1ms	remaining: 19.3s
5:	learn: 0.2702616	total: 113ms	remaining: 19.2s
6:	learn: 0.2524539	total: 132ms	remaining: 19.2s
7:	learn: 0.2395790	total: 154ms	remaining: 19.7s
8:	learn: 0.2305374	total: 181ms	remaining: 20.5s
9:	learn: 0.2232308	total: 205ms	remaining: 20.9s
10:	learn: 0.2186998	total: 228ms	remaining: 21s
11:	learn: 0.2135763	total: 247ms	remaining: 20.9s
12:	learn: 0.2096891	total: 266ms	remaining: 20.8s
13:	learn: 0.2071916	total: 285ms	remaining: 20.7s
14:	learn: 0.2048438	total: 304ms	remaining: 20.5s
15:	learn: 0.2027090	total: 322ms	remaining: 20.4s
16:	learn: 0.2006879	total: 340ms	remaining: 20.2s
17:	learn: 0.1991308	total: 360ms	remaining: 20.2s
18:	learn: 0.1973399	total: 379ms	remaining: 20.1s
19:	learn: 0.1955860	total: 398ms	remain

In [ ]:
model1 = XGBClassifier(**xgb_params)
model2 = RandomForestClassifier(**rf_params)
model3 = LGBMClassifier(**lgbm_params)
model4 = CatBoostClassifier(**cb_params)

model = VotingClassifier(estimators=[('xgb', model1), ('rf', model2), ('lgbm', model3), ('cb', model4)], voting='soft')

model.fit(X, y)

[LightGBM] [Info] Number of positive: 8350, number of negative: 50295
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1108
[LightGBM] [Info] Number of data points in the train set: 58645, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142382 -> initscore=-1.795644
[LightGBM] [Info] Start training from score -1.795644
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.5246249337091624,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=0.9778592845089441,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_...
                                                     min_samples_split=20,
                                                     n_estimators=269)),
                             ('lgbm',
                              LGBMClassifier(colsample_bytree=0.6113458989028802,
                                             learning_rate=0.08245968867808884,
                                             max_depth=6, min_child_samples=46,
                                             n_estimators=407, num_leaves=225,
                                             reg_alpha=7.697039904785172,
                                             reg_lambda=5.488324394636368,
                                             subsample=0.5469000974065992)),
                             ('cb',
                              <catboost.core.CatBoostClassifier object at 0x7dcc43731510>)],
                 voting='soft')

In [42]:
model = CatBoostClassifier(**cb_params)
model.fit(X, y)

0:	learn: 0.5377510	total: 34.6ms	remaining: 35.5s
1:	learn: 0.4343574	total: 86.1ms	remaining: 44.2s
2:	learn: 0.3663006	total: 146ms	remaining: 49.9s
3:	learn: 0.3217309	total: 207ms	remaining: 53s
4:	learn: 0.2894958	total: 237ms	remaining: 48.5s
5:	learn: 0.2696239	total: 279ms	remaining: 47.5s
6:	learn: 0.2533976	total: 306ms	remaining: 44.7s
7:	learn: 0.2415561	total: 335ms	remaining: 42.7s
8:	learn: 0.2312553	total: 382ms	remaining: 43.3s
9:	learn: 0.2233985	total: 411ms	remaining: 41.9s
10:	learn: 0.2178351	total: 466ms	remaining: 43.1s
11:	learn: 0.2123667	total: 523ms	remaining: 44.3s
12:	learn: 0.2081999	total: 567ms	remaining: 44.2s
13:	learn: 0.2051222	total: 642ms	remaining: 46.5s
14:	learn: 0.2027626	total: 749ms	remaining: 50.5s
15:	learn: 0.2006894	total: 825ms	remaining: 52.2s
16:	learn: 0.1988258	total: 909ms	remaining: 54.1s
17:	learn: 0.1976030	total: 936ms	remaining: 52.5s
18:	learn: 0.1953620	total: 974ms	remaining: 51.7s
19:	learn: 0.1941436	total: 1.03s	remaini

In [43]:
y_pred = model.predict(X_test_pred)

submission_df = pd.DataFrame({'id': test['id'], 'loan_status': y_pred})

submission_df['loan_status'] = submission_df['loan_status'].astype(int)

submission_df.to_csv('submission.csv', index=False)

In [ ]:
# !kaggle competitions submit -c playground-series-s4e10 -f submission.csv -m "Message"

## The best model is Catboost without specification of parameters with a public score: 0.87585
